# Google Cloud SQL for PostgreSQL

> [Cloud SQL](https://cloud.google.com/sql) is a fully managed relational database service that offers high performance, seamless integration, and impressive scalability. It offers MySQL, PostgreSQL, and SQL Server database engines. Extend your database application to build AI-powered experiences leveraging Cloud SQL's Langchain integrations.

This notebook goes over how to use `Cloud SQL for PostgreSQL` to store checkpoints with the PostgresSaver class.

Learn more about the package on [GitHub](https://github.com/googleapis/langchain-google-cloud-sql-pg-python/).

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googleapis/langchain-google-cloud-sql-pg-python/blob/main/docs/chat_message_history.ipynb)

## Before You Begin

To run this notebook, you will need to do the following:

 * [Create a Google Cloud Project](https://developers.google.com/workspace/guides/create-project)
 * [Enable the Cloud SQL Admin API.](https://console.cloud.google.com/marketplace/product/google/sqladmin.googleapis.com)
 * [Create a Cloud SQL for PostgreSQL instance](https://cloud.google.com/sql/docs/postgres/create-instance)
 * [Create a Cloud SQL database](https://cloud.google.com/sql/docs/mysql/create-manage-databases)
 * [Add an IAM database user to the database](https://cloud.google.com/sql/docs/postgres/add-manage-iam-users#creating-a-database-user) (Optional)


### 🦜🔗 Library Installation
The integration lives in its own `langchain-google-cloud-sql-pg` package, so we need to install it.

In [ ]:
%pip install --upgrade --quiet langchain-google-cloud-sql-pg langchain-google-vertexai

**Colab only:** Uncomment the following cell to restart the kernel or use the button to restart the kernel. For Vertex AI Workbench you can restart the terminal using the button on top.

In [ ]:
# # Automatically restart kernel after installs so that your environment can access the new packages
# import IPython
#
# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### 🔐 Authentication
Authenticate to Google Cloud as the IAM user logged into this notebook in order to access your Google Cloud Project.

* If you are using Colab to run this notebook, use the cell below and continue.
* If you are using Vertex AI Workbench, check out the setup instructions [here](https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env).

In [ ]:
from google.colab import auth

auth.authenticate_user()

### ☁ Set Your Google Cloud Project
Set your Google Cloud project so that you can leverage Google Cloud resources within this notebook.

If you don't know your project ID, try the following:

* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113).

In [ ]:
# @markdown Please fill in the value below with your Google Cloud project ID and then run the cell.

PROJECT_ID = "my-project-id"  # @param {type:"string"}

# Set the project id
!gcloud config set project {PROJECT_ID}

### 💡 API Enablement
The `langchain-google-cloud-sql-pg` package requires that you [enable the Cloud SQL Admin API](https://console.cloud.google.com/flows/enableapi?apiid=sqladmin.googleapis.com) in your Google Cloud Project.

In [ ]:
# enable Cloud SQL Admin API
!gcloud services enable sqladmin.googleapis.com

## Basic Usage

### Set Cloud SQL database values
Find your database values, in the [Cloud SQL Instances page](https://console.cloud.google.com/sql?_ga=2.223735448.2062268965.1707700487-2088871159.1707257687).

In [ ]:
# @title Set Your Values Here { display-mode: "form" }
REGION = "us-central1"  # @param {type: "string"}
INSTANCE = "my-postgresql-instance"  # @param {type: "string"}
DATABASE = "my-database"  # @param {type: "string"}
TABLE_NAME = "checkpoint"  # @param {type: "string"}

### PostgresEngine Connection Pool

One of the requirements and arguments to establish Cloud SQL as a PostgresSaver is a `PostgresEngine` object. The `PostgresEngine`  configures a connection pool to your Cloud SQL database, enabling successful connections from your application and following industry best practices.

To create a `PostgresEngine` using `PostgresEngine.from_instance()` you need to provide only 4 things:

1.   `project_id` : Project ID of the Google Cloud Project where the Cloud SQL instance is located.
1. `region` : Region where the Cloud SQL instance is located.
1. `instance` : The name of the Cloud SQL instance.
1. `database` : The name of the database to connect to on the Cloud SQL instance.

By default, [IAM database authentication](https://cloud.google.com/sql/docs/postgres/iam-authentication#iam-db-auth) will be used as the method of database authentication. This library uses the IAM principal belonging to the [Application Default Credentials (ADC)](https://cloud.google.com/docs/authentication/application-default-credentials) sourced from the envionment.

For more informatin on IAM database authentication please see:
* [Configure an instance for IAM database authentication](https://cloud.google.com/sql/docs/postgres/create-edit-iam-instances)
* [Manage users with IAM database authentication](https://cloud.google.com/sql/docs/postgres/add-manage-iam-users)

Optionally, [built-in database authentication](https://cloud.google.com/sql/docs/postgres/built-in-authentication) using a username and password to access the Cloud SQL database can also be used. Just provide the optional `user` and `password` arguments to `PostgresEngine.from_instance()`:

* `user` : Database user to use for built-in database authentication and login
* `password` : Database password to use for built-in database authentication and login.

In [ ]:
from langchain_google_cloud_sql_pg import PostgresEngine

engine = PostgresEngine.from_instance(
    project_id=PROJECT_ID, region=REGION, instance=INSTANCE, database=DATABASE
)

### Initialize a table
The `PostgresSaver` class requires a database table with a specific schema in order to store the persist LangGraph agents state.

The `PostgresEngine` engine has a helper method `init_checkpoint_table()` that can be used to create a table with the proper schema for you.

In [ ]:
engine.init_checkpoint_table(table_name=TABLE_NAME)

#### Optional Tip: 💡
You can also specify a schema name by passing `schema_name` wherever you pass `table_name`. Eg:

```python
SCHEMA_NAME="my_schema"

engine.init_chat_history_table(
    table_name=TABLE_NAME,
    schema_name=SCHEMA_NAME     # Default: "public"
)
```

### PostgresSaver

To initialize the `PostgresSaver` class you need to provide only 3 things:

1. `engine` - An instance of a `PostgresEngine` engine.
2. `table_name` : The name of the table within the Cloud SQL database to store the checkpoints (Default: "checkpoints").
3. `schema_name` : The name of the database schema containing the checkpoints table (Default: "public")
4. `serde`: Serializer for encoding/decoding checkpoints (Default: None)


In [ ]:
from langchain_google_cloud_sql_pg import PostgresSaver

checkpointer = PostgresSaver.create_sync(
        engine,
        table_name = TABLE_NAME
    )

write_config =  {"configurable": {"thread_id": "1", "checkpoint_ns": ""}}
read_config = {"configurable": {"thread_id": "1"}}

checkpoint = {
        "v": 1,
        "ts": "2024-07-31T20:14:19.804150+00:00",
        "id": "1ef4f797-8335-6428-8001-8a1503f9b875",
        "channel_values": {
            "my_key": "meow",
            "node": "node"
        },
        "channel_versions": {
            "__start__": 2,
            "my_key": 3,
            "start:node": 3,
            "node": 3
        },
        "versions_seen": {
            "__input__": {},
            "__start__": {
            "__start__": 1
            },
            "node": {
            "start:node": 2
            }
        },
        "pending_sends": [],
    }

# store checkpoint
checkpointer.put(write_config, checkpoint, {}, {})
# load checkpoint
checkpointer.get(read_config)

## 🔗 Adding persistence to the pre-built create react agent

### 🔧 Setup

Before running the project, ensure you configure the necessary environment variables properly.

#### Setting Up Environment Variables


## Up LangSmith for LangGraph Development**




In [ ]:
import getpass
import os

def _set_env(var: str):
    """Prompts the user for an environment variable if not already set."""
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

# Set OpenAI API key
_set_env("OPENAI_API_KEY")


[LangSmith](https://docs.langsmith.com/) is a tool designed to identify issues and improve the performance of LangGraph-based projects.

### **Getting Started with LangSmith**

1. **Sign up for LangSmith**: If you haven't already, create an account at [LangSmith](https://smith.langchain.com/).
2. **Set up LangSmith** in your development environment.
3. **Integrate LangSmith into your code** to enable trace collection and analysis.

For detailed instructions, refer to the [official LangSmith documentation](https://docs.langsmith.com/).

In [ ]:
from typing import Literal

from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent


@tool
def get_weather(city: Literal["nyc", "sf"]):
    """Use this to get weather information."""
    if city == "nyc":
        return "It might be cloudy in nyc"
    elif city == "sf":
        return "It's always sunny in sf"
    else:
        raise AssertionError("Unknown city")


tools = [get_weather]
model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

graph = create_react_agent(model, tools=tools, checkpointer=checkpointer)
config = {"configurable": {"thread_id": "2"}}
res = graph.invoke({"messages": [("human", "what's the weather in sf")]}, config)
checkpoint = checkpointer.get(config)

In [ ]:
res

In [ ]:
checkpoint